In [1]:
import gzip
import json
from pathlib import Path
from pprint import pprint

import requests
from tqdm.notebook import tqdm

In [2]:
import gzip
with gzip.open('ADS_DOI.tsv.gz') as f:
    ads_dois = [line.decode().strip().split("\t")[-1] for line in f]

In [3]:
print(f"{len(ads_dois):,} DOIs in ADS. Example: '{ads_dois[0]}'")

12,282,140 DOIs in ADS. Example: '10.3931/e-rara-1987'


In [4]:
base_uri = "https://api.openalex.org"
headers = {"User-Agent": "mailto:donny@polyneme.xyz"}

In [5]:
n_total = next(grp for grp in requests.get(
    base_uri + "/works",
    params={"group_by": "has_doi"},
    headers=headers,
).json()["group_by"] if grp["key"] == 'true')['count']
print(f"{n_total:,}", "DOIs in OpenAlex, so ratio of ADS/OpenAlex totals is",
      f"{len(ads_dois)/n_total:.1%}.",)

140,432,604 DOIs in OpenAlex, so ratio of ADS/OpenAlex totals is 8.7%.


## OpenAlex-snapshot coverage of ADS DOIs

In [6]:
!du -hs /data/openalex-snapshot/

323G	/data/openalex-snapshot/


In [7]:
!head /data/openalex-snapshot/RELEASE_NOTES.txt

OPENALEX STANDARD-FORMAT SNAPSHOT RELEASE NOTES

RELEASE 2022-10-10
- moved to the V3 tagger API giving complete paths to concept level 0
- restored 170M "lost" citations that were in MAG and we deleted
- added new works

RELEASE 2022-09-16
- added 1.3 million new Works
- removed 700 thousand duplicate Authors


In [8]:
works_parts = sorted(Path("/data/openalex-snapshot/data/works/").glob("**/*.gz"))
len(works_parts)

344

In [9]:
with open("/data/openalex-snapshot/data/works/manifest") as f:
    manifest = json.load(f)
    record_count = manifest["meta"]["record_count"]
record_count

243249262

In [10]:
ads_dois_set = set(ads_dois)

In [18]:
from pymongo import MongoClient
client = MongoClient(host="host.docker.internal")

In [20]:
docs = []

client.openalex.drop_collection("works")

pbar = tqdm(total=record_count)
for part in works_parts:
    with gzip.open(part, 'rb') as f:
        for line in f:
            doc = json.loads(line)
            if isinstance(doc.get("doi"), str) and doc["doi"][16:] in ads_dois_set:
                docs.append(doc)
            pbar.update(1)
            if len(docs) == 10000:
                print("dumping...")
                client.openalex.works.insert_many(docs)
                docs = []

  0%|          | 0/243249262 [00:00<?, ?it/s]

dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...
dumping...

# Indexing in ElasticSearch

In [3]:
import os

from elasticsearch import Elasticsearch


ELASTIC_PASSWORD = os.environ.get("ELASTIC_PASSWORD")

ModuleNotFoundError: No module named 'elasticsearch'